In [1]:
import os
import pandas as pd
from decimal import Decimal

In [2]:
# Capactiy_list file generation
folder_path = 'Raw data'
output_file_path = 'capacity list.xlsx'
columns = ['batch', 'sequence'] + [str(i) for i in range(0, 625, 25)]
capacity_df = pd.DataFrame(columns=columns)

file_list = sorted([f for f in os.listdir(folder_path) if f.endswith('.csv')])

for file in file_list:
    file_path = os.path.join(folder_path, file)
    data = pd.read_csv(file_path)
    
    batch = 'b' + file[-5]
    sequence = file[:7]
    row_data = [batch, sequence]
    
    cycle_points = list(range(4, 725, 30))
    for cycle in cycle_points:
        discharge_data = data[(data['Type'] == 'discharge') & (data['TotCycle'] == cycle)]
        capacity_value = discharge_data['Dischar. Cap.(Ah)'].iloc[-1] if not discharge_data.empty else None
        row_data.append(capacity_value)

    capacity_df.loc[len(capacity_df)] = row_data

capacity_df.to_excel(output_file_path, index=False)


In [ ]:
### Diagnosis folder generation
input_folder_path = 'Raw data'
output_folder_path = 'diagnosis'

if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

file_list = sorted([f for f in os.listdir(input_folder_path) if f.endswith('.csv')])

for file in file_list:
    input_file_path = os.path.join(input_folder_path, file)
    data = pd.read_csv(input_file_path, float_precision='high')

    new_file_name = file[:2] + '_b' + file[-5]
    output_file_path = os.path.join(output_folder_path, f'{new_file_name}.csv')

    totcycle_points = [2, 32, 62, 92, 122, 152, 182, 212, 242, 272, 302, 332, 362, 392, 422, 452, 482, 512, 542, 572, 602, 632, 662, 692, 722]

    stepno_mapping = {
        "range_1": [6, 8, 10, 12, 14, 16, 18, 20, 22, 24],
        "range_2": [49, 51, 53, 55, 57, 59, 61, 63, 65, 67],
        "range_3": [92, 94, 96, 98, 100, 102, 104, 106, 108, 110],
        "range_4": [135, 137, 139, 141, 143, 145, 147, 149, 151, 153],
        "range_5": [178, 180, 182, 184, 186, 188, 190, 192, 194, 196]
    }
    result_data = []

    for cycle in totcycle_points:
        if cycle == 2:
            stepno_list = stepno_mapping["range_1"]
        elif cycle in [32, 152, 272, 392, 512, 632]:
            stepno_list = stepno_mapping["range_2"]
        elif cycle in [62, 182, 302, 422, 542, 662]:
            stepno_list = stepno_mapping["range_3"]
        elif cycle in [92, 212, 332, 452, 572, 692]:
            stepno_list = stepno_mapping["range_4"]
        elif cycle in [122, 242, 362, 482, 602, 722]:
            stepno_list = stepno_mapping["range_5"]
        filtered_data = data[(data['TotCycle'] == cycle) & (data['Type'] == 'rest') & (data['StepNo'].isin(stepno_list))]

        voltage_diffs = []
        for _, group in filtered_data.groupby('StepNo'):
            v1 = Decimal(str(group['Voltage(V)'].iloc[-1]))
            v2 = Decimal(str(group['Voltage(V)'].iloc[1]))
            voltage_diffs.append(float(v1 - v2))

        result_data.append(voltage_diffs)

    result_df = pd.DataFrame(result_data, columns=[f'Step_{i}' for i in range(6, 26, 2)], index=totcycle_points)
    result_df.to_csv(output_file_path, header=False, index=False, float_format='%.4f')
    print(f"'{output_file_path}' has been saved.")